In [4]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.layers import  Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score
import numpy as np

Matplotlib is building the font cache; this may take a moment.


In [ ]:
pip install numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 1.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.4/22.4 MB 2.8 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
# Define image dimensions and batch size
img_height, img_width = 128, 128
batch_size = 20

# Define directories for training and testing data
train_data_dir = "../dataset/train"
test_data_dir = "../dataset/test"

In [6]:
# Data augmentation for training images
train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rescale=1./255  # Normalize pixel values
)

# Data augmentation for testing images (only rescale)
test_datagen = ImageDataGenerator(rescale=1./255)


In [7]:

# Create data generators for training and testing
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=5,
    class_mode='categorical',
    shuffle=False  # No need to shuffle for evaluation
)

Found 1174 images belonging to 14 classes.
Found 428 images belonging to 14 classes.


In [8]:

# Create the classification head
model = Sequential([
   # Convolution Layer 1
    Conv2D(32, (3, 3), input_shape=(img_height,img_width,3), activation='relu'),
    # MaxPooling2D(pool_size=(2, 2)),
    MaxPooling2D(pool_size=(2, 2)),

    # Convolution Layer 2
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # Convolution Layer 3
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # Flattening
    Flatten(),

    # Fully Connected Layer 1
    Dense(128, activation='relu'),
    # Dropout(0.5),

    # Fully Connected Layer 2
    Dense(64, activation='relu'),

    # Output Layer
    Dense(14, activation='sigmoid')  
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 14)             │           910 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,313,806 (12.64 MB)

 Trainable params: 3,313,806 (12.64 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=15,
    validation_data=test_generator,
    validation_steps=len(test_generator)
)




/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
59/59 ━━━━━━━━━━━━━━━━━━━━ 7s 105ms/step - accuracy: 0.0892 - loss: 2.7165 - val_accuracy: 0.3949 - val_loss: 2.0485
Epoch 2/15
59/59 ━━━━━━━━━━━━━━━━━━━━ 6s 104ms/step - accuracy: 0.2009 - loss: 2.2848 - val_accuracy: 0.3248 - val_loss: 1.8636
Epoch 3/15
59/59 ━━━━━━━━━━━━━━━━━━━━ 6s 105ms/step - accuracy: 0.2776 - loss: 1.9677 - val_accuracy: 0.2570 - val_loss: 1.8925
Epoch 4/15
59/59 ━━━━━━━━━━━━━━━━━━━━ 6s 104ms/step - accuracy: 0.3374 - loss: 1.7775 - val_accuracy: 0.4136 - val_loss: 1.5812
Epoch 5/15
59/59 ━━━━━━━━━━━━━━━━━━━━ 6s 105ms/step - accuracy: 0.4008 - loss: 1.6280 - val_accuracy: 0.3832 - val_loss: 1.5351
Epoch 6/15
59/59 ━━━━━━━━━━━━━━━━━━━━ 6s 108ms/step - accuracy: 0.4676 - loss: 1.4563 - val_accuracy: 0.4182 - val_loss: 1.5382
Epoch 7/15
59/59 ━━━━━━━━━━━━━━━━━━━━ 6s 108ms/step - accuracy: 0.5241 - loss: 1.3659 - val_accuracy: 0.4019 - val_loss: 1.5952
Epoch 8/15
59/59 ━━━━━━━━━━━━━━━━━━━━ 6s 107ms/step - accuracy: 0.4890 - loss: 1.3168 - val_accuracy: 0.

In [10]:
# predicting train dataset 
test_pridiction = model.predict(test_generator)

# true labels 
test_true_labels = test_generator.classes




86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step


In [11]:
print( test_pridiction.shape)
print( test_true_labels.shape)




(428, 14)
(428,)


In [12]:
#calculating accuracy
accuracy = accuracy_score(test_true_labels, np.argmax(test_pridiction, axis=1))
print(accuracy)

0.6098130841121495
